In [ ]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Plots, StatsPlots, Distributions

## ____________________________________________________________________

## Import Datasets (Outputted Probabilities and Returns on $100 Bets)

In [ ]:
# # add actual data here
# xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\xgb_cv_pred.csv",DataFrame)
# rf_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\rf_pred.csv",DataFrame)
# lgbm_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\lgbm_cv_pred.csv",DataFrame);
# logreg_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\logreg_pred.csv",DataFrame);
# new_xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\new_xgb_pred.csv",DataFrame);
# rf_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\rf_pred.csv",DataFrame);
# returns = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\data\\group_stage_betting_odds_final_cleaned.csv",DataFrame);

# # add actual data here
new_xgb_preds = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/notebooks/new_xgb_pred.csv",DataFrame);
# rf_preds = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/notebooks/rf_pred.csv",DataFrame);
returns = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/data/group_stage_betting_odds_final_cleaned.csv",DataFrame);

# real outcomes for each match (DONT NEED TO MANUALLY KEY IN THE REAL OUTCOMES OF EVERY MATCH)
real_outcomes = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/data/group_stage_mapped_outcomes.csv",DataFrame);

In [ ]:
returns_matrix = Matrix(returns[:,3:5]);
# # matrix of probabilities for model's predictions
# xgb_probs = Matrix(xgb_preds[:,3:5]);
# rf_probs = Matrix(rf_preds[:,3:5]);
# lgbm_probs = Matrix(lgbm_preds[:,3:5]);
# logreg_probs = Matrix(logreg_preds[:,3:5]);
new_xgb_probs = Matrix(new_xgb_preds[:,3:5]);

In [ ]:
new_xgb_preds[1:8,:]

Row,home_team,away_team,home_win,draw,home_loss
,String15,String15,Float64,Float64,Float64
1,Argentina,Mexico,0.691677,0.0415087,0.266814
2,Argentina,Saudi Arabia,0.968919,0.025733,0.00534816
3,Australia,Denmark,0.238239,0.210197,0.551563
4,Belgium,Canada,0.964543,0.0154218,0.0200349
5,Belgium,Morocco,0.86645,0.0244761,0.109074
6,Brazil,Serbia,0.760576,0.0116859,0.227739
7,Brazil,Switzerland,0.907287,0.00921022,0.0835025
8,Cameroon,Brazil,0.00561122,0.175407,0.818982


In [ ]:
returns[1:5,:]

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,Mexico,0.549451,2.95,6.12,0.64539,0.253165,0.140449
2,Argentina,Saudi Arabia,0.120048,8.21,24.52,0.892819,0.108578,0.039185
3,Australia,Denmark,6.38,3.88,0.429185,0.135501,0.204918,0.6997
4,Belgium,Canada,0.900901,2.45,3.51,0.526066,0.289855,0.221729
5,Belgium,Morocco,0.581395,3.21,4.92,0.632353,0.23753,0.168919


In [ ]:
# check if each row of new_xgb_preds and returns are the same
for i in 1:size(new_xgb_preds,1)
    if new_xgb_preds[i,1] != returns[i,1] || new_xgb_preds[i,2] != returns[i,2]
        println("XGB and Returns not the same")
    end
end

# check if each row of new_xgb_preds and real_outcomes are the same
for i in 1:size(new_xgb_preds,1)
    if new_xgb_preds[i,1] != real_outcomes[i,1] || new_xgb_preds[i,2] != real_outcomes[i,2]
        println("XGB and Real Outcomes not the same")
    end
end

# check if returns and real_outcomes are the same
for i in 1:size(returns,1)
    if returns[i,1] != real_outcomes[i,1] || returns[i,2] != real_outcomes[i,2]
        println("Returns and Real Outcomes not the same")
    end
end

## ____________________________________________________________________

In [ ]:
function FIFA2022_GS_Bettings(P, R, B, K)
    
    # define model
    model = Model(Gurobi.Optimizer)

    # define parameters and decision variables
    m = size(P, 1)              # 48 group stage matches
    n = size(P, 2)              # 3 outcomes

    @variable(model, x[i=1:m, j=1:n] >= 0) # x[i,j] is the amount of money bet on outcome j of match i
    @variable(model, z[i=1:m, j=1:n], Bin) # z[i,j] is whether you bet on outcome j of match i

    # define objective: maximising expected total returns across all matches
    @objective(model, Max, sum(P[i,1]*R[i,1]*x[i,1] + 
                               P[i,2]*R[i,2]*x[i,2] + 
                               P[i,3]*R[i,3]*x[i,3] - 
                               sum(x[i,j] for j=1:n) for i=1:m))

    # # constraints
    # 1) You can only bet on one outcome for each match
    @constraint(model, [i=1:m], sum(z[i,j] for j=1:n) <= 1)

    # 2) x[i,j] <= B/12*z[i,j] for all i and j
    # you can't bet more than 5% of your budget on a single match
    @constraint(model, [i=1:m, j=1:n], x[i,j] <= B/20*z[i,j])

    # 3) Budget constraints
    @constraint(model, sum(x[i,j] for j=1:n, i=1:m) <= B)

    # 4) You must spread your risk across at least K matches
    @constraint(model, sum(z[i,j] for j=1:n, i=1:m) >= K)

    # 5) if z = 1, then x > 0
    @constraint(model, [i=1:m, j=1:n], x[i,j] >= 1*z[i,j])

    # solve model
    optimize!(model)

    # get optimal bet amounts (x) and how you bet (z)
    opt_val = objective_value(model)
    x_opt = value.(x)
    z_opt = value.(z)

    return opt_val, x_opt, z_opt
end

FIFA2022_GS_Bettings (generic function with 1 method)

In [ ]:
exp_prof, x_opt, z_opt = FIFA2022_GS_Bettings(new_xgb_probs, returns_matrix, 5000, 5);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 338 rows, 288 columns and 1008 nonzeros
Model fingerprint: 0xa1ebd13e
Variable types: 144 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e-02, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 119.8196463
Presolve removed 338 rows and 288 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 5220.83 119.82 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.220830380900e+03, best bound 5.220830380900e+03, gap 0.0000%

User-callback calls 520, time in 

In [ ]:
# which matches to bet on
betting_matches = findall(x -> x >= 1, z_opt)

18-element Vector{CartesianIndex{2}}:
 CartesianIndex(3, 1)
 CartesianIndex(22, 1)
 CartesianIndex(28, 1)
 CartesianIndex(37, 1)
 CartesianIndex(38, 1)
 CartesianIndex(11, 2)
 CartesianIndex(21, 2)
 CartesianIndex(31, 2)
 CartesianIndex(35, 2)
 CartesianIndex(39, 2)
 CartesianIndex(41, 2)
 CartesianIndex(47, 2)
 CartesianIndex(1, 3)
 CartesianIndex(6, 3)
 CartesianIndex(12, 3)
 CartesianIndex(15, 3)
 CartesianIndex(19, 3)
 CartesianIndex(45, 3)

In [ ]:
# make betting matches into list of first index of tuples
betting_matches_list = [x[1] for x in betting_matches]
betting_matches_outcomes = [x[2] for x in betting_matches]

betting_matches_list

18-element Vector{Int64}:
  3
 22
 28
 37
 38
 11
 21
 31
 35
 39
 41
 47
  1
  6
 12
 15
 19
 45

In [ ]:
new_xgb_preds[3,:]

Row,home_team,away_team,home_win,draw,home_loss
,String15,String15,Float64,Float64,Float64
3,Australia,Denmark,0.238239,0.210197,0.551563


In [ ]:
x_opt[3,:]

3-element Vector{Float64}:
 250.0
   0.0
   0.0

In [ ]:
returns[11,:]

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
11,Costa Rica,Germany,22.79,10.17,0.1,0.0420345,0.0895255,0.909091


In [ ]:
outcomes = new_xgb_preds[betting_matches_list,:]

Row,home_team,away_team,home_win,draw,home_loss
,String15,String15,Float64,Float64,Float64
1,Australia,Denmark,0.238239,0.210197,0.551563
2,Iran,USA,0.515694,0.270426,0.21388
3,Morocco,Croatia,0.465358,0.434983,0.0996595
4,Saudi Arabia,Mexico,0.557936,0.14692,0.295144
5,Senegal,Netherlands,0.246266,0.403616,0.350118
6,Costa Rica,Germany,0.0131862,0.444052,0.542761
7,Ghana,Uruguay,0.090158,0.393147,0.516695
8,Poland,Argentina,0.0124997,0.398129,0.589371
9,Qatar,Ecuador,0.326202,0.570737,0.103061


In [ ]:
outcomes[:,"bet_amount"] = [x_opt[i] for i in betting_matches];

In [ ]:
# # rf_pred_outcomes
# outcomes[:,"actual_outcome"] = [1,3,3,2,1];

# # new_xgb_pred_outcomes (fill out)
# outcomes[:,"actual_outcome"]=real_

# take real match outcomes from the real_outcomes dataframe by taking the corresponding indices from betting_matches_list
outcomes[:,"actual_outcome"] = [real_outcomes[i,4] for i in betting_matches_list];

In [ ]:
outcomes[:,"bet_outcome"] = [x[2] for x in betting_matches];

In [ ]:
# is actual_outcome = bet_outcome?
outcomes[:,"correct"] = outcomes[:,"actual_outcome"] .== outcomes[:,"bet_outcome"];

In [ ]:
# bet result: if correct == false, then bet_result = -bet_amount, if correct == true, then bet_result = bet_amount*
bet_result = exp_prof - sum(outcomes[:,"bet_amount"].* (outcomes[:,"correct"].==false))

1470.8303809000008

In [ ]:
exp_prof-sum(outcomes[:,"bet_amount"].*(outcomes[:,"correct"].==false))

1470.8303809000008

In [ ]:
outcomes

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool
1,Australia,Denmark,0.238239,0.210197,0.551563,250.0,1,1,true
2,Iran,USA,0.515694,0.270426,0.21388,250.0,3,1,false
3,Morocco,Croatia,0.465358,0.434983,0.0996595,250.0,2,1,false
4,Saudi Arabia,Mexico,0.557936,0.14692,0.295144,250.0,3,1,false
5,Senegal,Netherlands,0.246266,0.403616,0.350118,250.0,3,1,false
6,Costa Rica,Germany,0.0131862,0.444052,0.542761,250.0,3,2,false
7,Ghana,Uruguay,0.090158,0.393147,0.516695,250.0,3,2,false
8,Poland,Argentina,0.0124997,0.398129,0.589371,250.0,3,2,false
9,Qatar,Ecuador,0.326202,0.570737,0.103061,250.0,3,2,false


In [ ]:
betting_matches

18-element Vector{CartesianIndex{2}}:
 CartesianIndex(3, 1)
 CartesianIndex(22, 1)
 CartesianIndex(28, 1)
 CartesianIndex(37, 1)
 CartesianIndex(38, 1)
 CartesianIndex(11, 2)
 CartesianIndex(21, 2)
 CartesianIndex(31, 2)
 CartesianIndex(35, 2)
 CartesianIndex(39, 2)
 CartesianIndex(41, 2)
 CartesianIndex(47, 2)
 CartesianIndex(1, 3)
 CartesianIndex(6, 3)
 CartesianIndex(12, 3)
 CartesianIndex(15, 3)
 CartesianIndex(19, 3)
 CartesianIndex(45, 3)

## money actually earned

In [ ]:
outcomes

# add betting_matches_list as a column called :betting_indices so we can map to the correct row and columns in 'returns' dataframe
outcomes[:,"betting_matches_indices"] = betting_matches_list;
outcomes[:,"betting_matches_outcomes"] = betting_matches_outcomes;
outcomes

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,betting_matches_outcomes
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Int64
1,Australia,Denmark,0.238239,0.210197,0.551563,250.0,1,1,true,3,1
2,Iran,USA,0.515694,0.270426,0.21388,250.0,3,1,false,22,1
3,Morocco,Croatia,0.465358,0.434983,0.0996595,250.0,2,1,false,28,1
4,Saudi Arabia,Mexico,0.557936,0.14692,0.295144,250.0,3,1,false,37,1
5,Senegal,Netherlands,0.246266,0.403616,0.350118,250.0,3,1,false,38,1
6,Costa Rica,Germany,0.0131862,0.444052,0.542761,250.0,3,2,false,11,2
7,Ghana,Uruguay,0.090158,0.393147,0.516695,250.0,3,2,false,21,2
8,Poland,Argentina,0.0124997,0.398129,0.589371,250.0,3,2,false,31,2
9,Qatar,Ecuador,0.326202,0.570737,0.103061,250.0,3,2,false,35,2


In [ ]:
# # DETERMINISTIC IF BETS ARE CORRECT PROFIT
# create a new column called "if_bets_correct_profit" that calculates the profit if the bets are correct
# initialise new if bets correct profit list
if_bets_correct_profit = zeros(size(outcomes,1))

# for each row in outcomes, calculate the profit if the bets are correct
for i in 1:size(outcomes,1)
    if_bets_correct_profit[i] = outcomes[i,"bet_amount"] * returns[outcomes[i,"betting_matches_indices"],outcomes[i,"betting_matches_outcomes"]+2] - outcomes[i,"bet_amount"]
end

# add if_bets_correct_profit to outcomes dataframe
outcomes[:,"if_bets_correct_profit"] = if_bets_correct_profit;
outcomes


Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,betting_matches_outcomes,if_bets_correct_profit
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Int64,Float64
1,Australia,Denmark,0.238239,0.210197,0.551563,250.0,1,1,true,3,1,1345.0
2,Iran,USA,0.515694,0.270426,0.21388,250.0,3,1,false,22,1,545.0
3,Morocco,Croatia,0.465358,0.434983,0.0996595,250.0,2,1,false,28,1,392.5
4,Saudi Arabia,Mexico,0.557936,0.14692,0.295144,250.0,3,1,false,37,1,835.0
5,Senegal,Netherlands,0.246266,0.403616,0.350118,250.0,3,1,false,38,1,1022.5
6,Costa Rica,Germany,0.0131862,0.444052,0.542761,250.0,3,2,false,11,2,2292.5
7,Ghana,Uruguay,0.090158,0.393147,0.516695,250.0,3,2,false,21,2,455.0
8,Poland,Argentina,0.0124997,0.398129,0.589371,250.0,3,2,false,31,2,642.5
9,Qatar,Ecuador,0.326202,0.570737,0.103061,250.0,3,2,false,35,2,400.0


In [ ]:
# # ACTUAL PROFIT
# for each row i in outcomes:
# if correct == true, then actual_profit = if_bets_correct_profit, 
# if correct == false, then actual_profit = -bet_amount

# initialise new actual profit list
actual_profit = zeros(size(outcomes,1))

# for each row in outcomes, calculate the actual profit and store in actual_profit list
for i in 1:size(outcomes,1)
    if outcomes[i,"correct"] == true
        actual_profit[i] = outcomes[i,"if_bets_correct_profit"]
    else
        actual_profit[i] = -outcomes[i,"bet_amount"]
    end
end

# add actual_profit to outcomes dataframe
outcomes[:,"actual_profit"] = actual_profit;
outcomes

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,betting_matches_outcomes,if_bets_correct_profit,actual_profit
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Int64,Float64,Float64
1,Australia,Denmark,0.238239,0.210197,0.551563,250.0,1,1,true,3,1,1345.0,1345.0
2,Iran,USA,0.515694,0.270426,0.21388,250.0,3,1,false,22,1,545.0,-250.0
3,Morocco,Croatia,0.465358,0.434983,0.0996595,250.0,2,1,false,28,1,392.5,-250.0
4,Saudi Arabia,Mexico,0.557936,0.14692,0.295144,250.0,3,1,false,37,1,835.0,-250.0
5,Senegal,Netherlands,0.246266,0.403616,0.350118,250.0,3,1,false,38,1,1022.5,-250.0
6,Costa Rica,Germany,0.0131862,0.444052,0.542761,250.0,3,2,false,11,2,2292.5,-250.0
7,Ghana,Uruguay,0.090158,0.393147,0.516695,250.0,3,2,false,21,2,455.0,-250.0
8,Poland,Argentina,0.0124997,0.398129,0.589371,250.0,3,2,false,31,2,642.5,-250.0
9,Qatar,Ecuador,0.326202,0.570737,0.103061,250.0,3,2,false,35,2,400.0,-250.0


In [ ]:
# sum outcomes[:,"actual_earnings"]
sum(outcomes[:,"actual_profit"])

-1780.0

## _________________________________________________________

## <u>Looping to Find The Optimal Betting Percentage

In [ ]:
function FIFA2022_GS_Bettings_LOOP(P, R, B, K, perc)
    
    # define divisor
    divisor = 1/perc

    # define model
    model = Model(Gurobi.Optimizer)

    # define parameters and decision variables
    m = size(P, 1)              # 48 group stage matches
    n = size(P, 2)              # 3 outcomes

    @variable(model, x[i=1:m, j=1:n] >= 0) # x[i,j] is the amount of money bet on outcome j of match i
    @variable(model, z[i=1:m, j=1:n], Bin) # z[i,j] is whether you bet on outcome j of match i

    # define objective: maximising expected total returns across all matches
    @objective(model, Max, sum(P[i,1]*R[i,1]*x[i,1] + 
                               P[i,2]*R[i,2]*x[i,2] + 
                               P[i,3]*R[i,3]*x[i,3] - 
                               sum(x[i,j] for j=1:n) for i=1:m))

    # # constraints
    # 1) You can only bet on one outcome for each match
    @constraint(model, [i=1:m], sum(z[i,j] for j=1:n) <= 1)

    # 2) x[i,j] <= B/12*z[i,j] for all i and j
    # you can't bet more than 10% of your budget on a single match
    @constraint(model, [i=1:m, j=1:n], x[i,j] <= (B/divisor)*z[i,j])

    # 3) Budget constraints
    @constraint(model, sum(x[i,j] for j=1:n, i=1:m) <= B)

    # 4) You must spread your risk across at least K matches
    @constraint(model, sum(z[i,j] for j=1:n, i=1:m) >= K)

    # 5) if z = 1, then x > 0
    @constraint(model, [i=1:m, j=1:n], x[i,j] >= 1*z[i,j])

    # solve model
    optimize!(model)

    # get optimal bet amounts (x) and how you bet (z)
    opt_val = objective_value(model)
    x_opt = value.(x)
    z_opt = value.(z)

    return opt_val, x_opt, z_opt
end

FIFA2022_GS_Bettings_LOOP (generic function with 1 method)

In [ ]:
# create a list of perc values 0.01 to 1 in steps of 0.01
perc_list = [0.01:0.01:1;]

# initlaise an empty matrix to store the expected profit for each perc value
exp_prof_each_perc = zeros(length(perc_list),4)

for p = 1:length(perc_list)

    exp_prof, x_opt, z_opt = FIFA2022_GS_Bettings_LOOP(new_xgb_probs, returns_matrix, 5000, 5, perc_list[p]);

    # which matches to bet on
    betting_matches = findall(x -> x >= 1, z_opt)

    # make betting matches into list of first (which matches u bet on) and second (which outcome u bet on) index of tuples
    betting_matches_list = [x[1] for x in betting_matches]
    betting_matches_outcomes = [x[2] for x in betting_matches]

    # create new dataframe called outcomes
    outcomes = new_xgb_preds[betting_matches_list,:]

    # save betting amount
    outcomes[:,"bet_amount"] = [x_opt[i] for i in betting_matches];

    # take real match outcomes from the real_outcomes dataframe by taking the corresponding indices from betting_matches_list
    outcomes[:,"actual_outcome"] = [real_outcomes[i,4] for i in betting_matches_list];
    outcomes[:,"bet_outcome"] = [x[2] for x in betting_matches];

    # is actual_outcome = bet_outcome?
    outcomes[:,"correct"] = outcomes[:,"actual_outcome"] .== outcomes[:,"bet_outcome"];
    
    # add betting_matches_list as a column called :betting_indices so we can map to the correct row and columns in 'returns' dataframe
    outcomes[:,"betting_matches_indices"] = betting_matches_list;

    # # DETERMINISTIC IF BETS CORRECT PROFIT
    # create a new column called "if_bets_correct_profit" that calculates the profit if the bets are correct
    # initialise new if bets correct profit list
    if_bets_correct_profit = zeros(size(outcomes,1))

    # for each row in outcomes, calculate the profit if the bets are correct
    for i in 1:size(outcomes,1)
        if_bets_correct_profit[i] = outcomes[i,"bet_amount"] * returns[outcomes[i,"betting_matches_indices"],outcomes[i,"bet_outcome"]+2] - outcomes[i,"bet_amount"]
    end

    # add if_bets_correct_profit to outcomes dataframe
    outcomes[:,"if_bets_correct_profit"] = if_bets_correct_profit;
   
    # # ACTUAL PROFIT
    # for each row i in outcomes:
    # if correct == true, then actual_profit = if_bets_correct_profit, 
    # if correct == false, then actual_profit = -bet_amount

    # initialise new actual profit list
    actual_profit = zeros(size(outcomes,1))

    # for each row in outcomes, calculate the actual profit and store in actual_profit list
    for i in 1:size(outcomes,1)
        if outcomes[i,"correct"] == true
            actual_profit[i] = outcomes[i,"if_bets_correct_profit"]
        else
            actual_profit[i] = -outcomes[i,"bet_amount"]
        end
    end

    # add actual_profit to outcomes dataframe
    outcomes[:,"actual_profit"] = actual_profit;

    # print(perc) and sum outcomes[:,"actual_profit"]
    # save perc_list[p] into exp_prof_each_perc[p,1] and sum(outcomes[:,"actual_profit"]) into exp_prof_each_perc[p,2]
    exp_prof_each_perc[p,1] = perc_list[p]
    exp_prof_each_perc[p,2] = sum(outcomes[:,"actual_profit"])
    exp_prof_each_perc[p,3] = sum(outcomes[:,"if_bets_correct_profit"])
    exp_prof_each_perc[p,4] = exp_prof
end

    

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19

In [ ]:
# convert exp_prof_each_perc to dataframe
exp_prof_each_perc_df = DataFrame(exp_prof_each_perc, :auto)

# rename 1st column as %_of_budget_perMatch and 2nd column as actual_profit
rename!(exp_prof_each_perc_df, Dict(1 => "%_of_budget_perMatch", 
                                    2 => "total_actual_profit", 
                                    3 => "total_if_bets_correct_profit", 
                                    4 => "expected_model_profit"))

Row,%_of_budget_perMatch,total_actual_profit,total_if_bets_correct_profit,expected_model_profit
,Float64,Float64,Float64,Float64
1,0.01,-356.0,3113.5,1044.17
2,0.02,-712.0,6227.0,2088.33
3,0.03,-1068.0,9340.5,3132.5
4,0.04,-1424.0,12454.0,4176.66
5,0.05,-1780.0,15567.5,5220.83
6,0.06,-1736.0,18091.0,6244.07
7,0.07,-1192.0,19714.0,7180.45
8,0.08,-648.0,21034.0,8058.53
9,0.09,-1072.0,22175.0,8878.69


In [ ]:
# print row where % of budget per match is 0.083
exp_prof_each_perc_df[exp_prof_each_perc_df[!,1] .== 0.083,:]

# print row where actual_profit is max
exp_prof_each_perc_df[exp_prof_each_perc_df[!,3] .== maximum(exp_prof_each_perc_df[!,3]),:]

Row,%_of_budget_perMatch,total_actual_profit,total_if_bets_correct_profit,expected_model_profit
,Float64,Float64,Float64,Float64
1,1.0,-5000.0,59272.4,53100.7


In [ ]:
1/0.166

6.024096385542168

In [ ]:
outcomes

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,if_bets_correct_profit,actual_profit
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Float64,Float64
1,Iran,USA,0.515694,0.270426,0.21388,1.0,3,1,false,22,2.18,-1.0
2,Saudi Arabia,Mexico,0.557936,0.14692,0.295144,1.0,3,1,false,37,3.34,-1.0
3,Costa Rica,Germany,0.0131862,0.444052,0.542761,1.0,3,2,false,11,9.17,-1.0
4,Argentina,Mexico,0.691677,0.0415087,0.266814,1.0,1,3,false,1,5.12,-1.0
5,France,Denmark,0.0639191,0.0319283,0.904153,4996.0,1,3,false,19,59252.6,-4996.0
